In [1]:

%pip install nltk
%pip install spacy
nltk.download('punkt_tab')
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
stemmer = PorterStemmer()
Lemma = WordNetLemmatizer()
nltk.download('wordnet')
import spacy
import en_core_web_sm
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


NameError: name 'nltk' is not defined

In [ ]:


df = pd.read_csv(r"train(1).csv")
df

,title,text
0,"Port St. Lucie, Florida","Port St. Lucie is a city in St. Lucie County, ..."
1,Dirty Dozen,"Dirty Dozen may refer to:\n\nBooks, film and t..."
2,Glenrothes,"Glenrothes (; , ; ; ) is a town situated in th..."
3,Roughnecks: Starship Troopers Chronicles,Roughnecks: Starship Troopers Chronicles is a ...
4,June 13,\n\nEvents\n\nPre-1600\n 313 – The decisions o...
...,...,...
13874,"Northport, New York",Northport is a historic maritime village on th...
13875,Ko,"A KO is a knockout in various sports, such as ..."
13876,Rudy Ruettiger,"Daniel Eugene ""Rudy"" Ruettiger (born August 23..."
13877,Chicago Cubs,The Chicago Cubs are an American professional ...


In [140]:
df = df.text

In [141]:
from tqdm import tqdm
import spacy
from nltk import word_tokenize, sent_tokenize
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner","tok2vec"])

## Preprocessing

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

pre_corpus=[]
# # Load the English language model
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner","tok2vec"])

texts = list(df)

# Process the texts
for doc in tqdm(nlp.pipe(texts, batch_size=50)):
    # Perform lemmatization and stopword removal
    lemmatized_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
    
    # Join the tokens back into a string
    processed_text = " ".join(lemmatized_tokens)
    
    # Print the processed text
    pre_corpus.append(processed_text)



13879it [24:54,  9.29it/s]


In [143]:
import pandas as pd


# Convert list to DataFrame
df = pd.DataFrame(pre_corpus, columns=["Text"])

# Save as CSV
df.to_csv("pre_corpus.csv", index=False)  # Set index=False to avoid unnecessary index column

print("CSV file saved successfully!")

CSV file saved successfully!


In [ ]:
test_set = pd.read_csv("test(1).csv")
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
df_test = pd.read_csv("test(1).csv")
df_test=df_test.text
pre_corpus=[]

texts = list(df_test)

# Process the texts
for doc in tqdm(nlp.pipe(texts, batch_size=50)):
    # Perform lemmatization and stopword removal
    lemmatized_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
    
    # Join the tokens back into a string
    processed_text = " ".join(lemmatized_tokens)
    
    # Print the processed text
    pre_corpus.append(processed_text)




# Convert list to DataFrame
df = pd.DataFrame(pre_corpus, columns=["Text"])

# Save as CSV
df.to_csv("test_pre_corpus.csv", index=False)  # Set index=False to avoid unnecessary index column

print("CSV file saved successfully!")


100it [00:10,  9.14it/s]

CSV file saved successfully!


In [145]:
df2 = pd.read_csv("pre_corpus.csv")  # Ensure df2 is the correct dataset
corpus = df2["Text"].dropna().tolist() 

In [146]:
df2

,Text
0,port lucie city lucie county florida united st...
1,dirty dozen refer books film television dirty ...
2,glenrothes town situated heart fife east centr...
3,roughnecks starship troopers chronicles cgi an...
4,events decisions edict milan signed constantin...
...,...
13874,northport historic maritime village northern s...
13875,ko knockout sports boxing martial arts ko kō r...
13876,daniel eugene rudy ruettiger born august motiv...
13877,chicago cubs american professional baseball te...


## N-gram Modelling

In [ ]:

from collections import Counter, defaultdict
from tqdm import tqdm

def preprocess_text(text):
    return text.split()

def calculate_token_frequencies(corpus):
    # Flatten the list of sentences into a single list of tokens
    all_tokens = [token for sentence in corpus for token in sentence]
    return Counter(all_tokens)

def filter_vocabulary_by_threshold(corpus, threshold=0.01):
    # Calculate the token frequencies across the corpus
    token_frequencies = calculate_token_frequencies(corpus)
    
    # Calculate the total number of tokens in the corpus
    total_tokens = sum(token_frequencies.values())
    
    # Apply threshold filtering
    filtered_vocab = {token for token, freq in token_frequencies.items() if (freq ) >= (threshold*len(corpus))}
    
    return filtered_vocab, len(filtered_vocab)

def create_ngram_counts(corpus, n, filtered_vocab):
    ngram_counts = defaultdict(int)
    context_counts = defaultdict(int)
    
    for tokens in tqdm(corpus):
        # Only use tokens that are in the filtered vocabulary
        tokens = [token for token in tokens if token in filtered_vocab]
        
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i + n])
            context = tuple(tokens[i:i + n - 1])
            ngram_counts[ngram] += 1
            context_counts[context] += 1
    return ngram_counts, context_counts

def train_ngram_model(corpus, n, filtered_vocab,v):
    ngram_counts, context_counts = create_ngram_counts(corpus, n, filtered_vocab)
    
    probabilities = {}
    for ngram, count in tqdm(ngram_counts.items()):
        context = ngram[:-1]
        context_key = tuple(context) if context else ()
        prob = (count + 1) / (context_counts.get(context_key,0) + len(v))
        if count >= (0.01 * len(corpus)):  # 1% threshold
            probabilities[ngram] = prob
    
    return probabilities

def filter_ngram_dict(corpus, n, filtered_vocab):
    ngram_counts, context_counts = create_ngram_counts(corpus, n, filtered_vocab)
    
    filter_ngram = {}
    for ngram, count in tqdm(ngram_counts.items()):
        context = ngram[:-1]
        context_key = tuple(context) if context else ()
        if count >= (0.01 * len(corpus)):  # 1% threshold
            filter_ngram[ngram] = count

    return filter_ngram


corpus = df2[:-100].Text.dropna().tolist()
processed_corpus = [preprocess_text(para) for para in corpus]

# Filter vocabulary using a threshold (1% frequency threshold)
filtered_vocab, filtered_vocab_size = filter_vocabulary_by_threshold(processed_corpus, threshold=0.01)

#  n-gram counts using the filtered vocabulary
unigram_counts, unigram_context = create_ngram_counts(processed_corpus, 1, filtered_vocab)
bigram_counts, bigram_context = create_ngram_counts(processed_corpus, 2, filtered_vocab)
trigram_counts, trigram_context = create_ngram_counts(processed_corpus, 3, filtered_vocab)

# Get diffirent vocabulary Dictionaries
unigram_v=filter_ngram_dict(processed_corpus,1,filtered_vocab)
bigram_v = filter_ngram_dict(processed_corpus,2,filtered_vocab)
trigram_v = filter_ngram_dict(processed_corpus,3,filtered_vocab)


# Step 3: Train n-gram model with filtered vocabulary
unigram_probs = train_ngram_model(processed_corpus, 1, filtered_vocab,unigram_v)
bigram_probs = train_ngram_model(processed_corpus, 2, filtered_vocab,bigram_v)
trigram_probs = train_ngram_model(processed_corpus, 3, filtered_vocab,trigram_v)

print("Filtered Vocabulary Size:", filtered_vocab_size)
print("Sample Unigram Probabilities:", list(unigram_probs.items())[:5])
print("Sample Bigram Probabilities:", list(bigram_probs.items())[:5])
print("Sample Trigram Probabilities:", list(trigram_probs.items())[:5])


100%|██████████| 12427057/12427057 [00:20<00:00, 605748.96it/s]


Filtered Vocabulary Size: 12985
Sample Unigram Probabilities: [(('port',), 0.00035390552787373), (('city',), 0.005551092171740339), (('county',), 0.002033806125824797), (('florida',), 0.00043003564494182233), (('united',), 0.003233601843343998)]
Sample Bigram Probabilities: [(('port', 'city'), 0.01182168951646006), (('city', 'county'), 0.009549681329014731), (('county', 'florida'), 0.007023727428294128), (('united', 'states'), 0.487054960839974), (('county', 'population'), 0.008454013740964932)]
Sample Trigram Probabilities: [(('combined', 'statistical', 'area'), 0.09879839786381843), (('households', 'children', 'age'), 0.6559139784946236), (('children', 'age', 'living'), 0.6492867683226758), (('age', 'living', 'married'), 0.6236559139784946), (('living', 'married', 'couples'), 0.6504653567735263)]


## Perplexity

In [148]:
v_uni = unigram_v
v_bi = bigram_v
v_tri = trigram_v


In [154]:
import numpy as np
import spacy


def calculate_perplexity(test_corpus, ngram_probs, context_counts, n, vocab_size, k=1):
    sum_perp = 0

    for text in test_corpus:
       
        tokens = text.split()
        text_log_prob = 0
        text_words = len(tokens)
        ##  assign different vocabularies
        if(n==1):
            vocab_size= len(v_uni)
        elif(n==2):
            vocab_size= len(v_bi)
        else:
            vocab_size= len(v_tri)
        
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i + n])
            context = ngram[:-1]

            if ngram in ngram_probs:
                prob = ngram_probs[ngram]
            else:                       ## if that n-gram was not seen
                # Apply Laplace smoothing
                numerator = k
                denominator = context_counts.get(context, 0) + k * vocab_size
                prob = numerator / denominator

            text_log_prob += np.log(prob)

        if text_words > 0:
            avg_log_prob = text_log_prob / text_words
            perplexity = np.exp(-avg_log_prob)
            sum_perp += perplexity

    return sum_perp / len(test_corpus)

def compute_perplexities(df_test, unigram_probs, bigram_probs, trigram_probs, 
                         unigram_context, bigram_context, trigram_context, vocab_size):
    test_corpus = df_test.Text.dropna().tolist()

    unigram_perplexity = calculate_perplexity(test_corpus, unigram_probs, unigram_context, 1, v_uni)
    bigram_perplexity = calculate_perplexity(test_corpus, bigram_probs, bigram_context, 2, v_bi)
    trigram_perplexity = calculate_perplexity(test_corpus, trigram_probs, trigram_context, 3, v_tri)

    print(f"Unigram Model Perplexity: {unigram_perplexity:.3f}")
    print(f"Bigram Model Perplexity: {bigram_perplexity:.3f}")
    print(f"Trigram Model Perplexity: {trigram_perplexity:.3f}")

    return unigram_perplexity, bigram_perplexity, trigram_perplexity

# Example usage
perplexities = compute_perplexities(df2[-100:], unigram_probs, bigram_probs, trigram_probs, 
                                    unigram_context, bigram_context, trigram_context, vocab_size=v_uni)


Unigram Model Perplexity: 17105.445
Bigram Model Perplexity: 4525.801
Trigram Model Perplexity: 1110.731


In [150]:
validation_set=df2[-100:]
test_set = pd.read_csv("test_pre_corpus.csv")

In [151]:
test_set

,Text
0,weyburn eleventh largest city saskatchewan can...
1,catholic high school chs government aided auto...
2,minnesota golden gophers commonly shortened go...
3,following notable people born raised lived sig...
4,theobald germanic dithematic composed elements...
...,...
95,kutztown university pennsylvania kutztown univ...
96,paul william bear bryant september january ame...
97,jonathan michael paul spector born march ameri...
98,abia state state south east geopolitical zone ...


##  Interpolation Model

In [159]:
import numpy as np
from tqdm import tqdm

def compute_interpolated_prob(w1, w2, w3, weights):
    lambda1, lambda2, lambda3 = weights
    
    trigram_prob = trigram_probs.get((w1, w2, w3), 1 / len(v_tri))
    bigram_prob = bigram_probs.get((w2, w3), 1 / len(v_bi))
    unigram_prob = unigram_probs.get((w3,), 1 / len(v_uni))
    
    return lambda1 * trigram_prob + lambda2 * bigram_prob + lambda3 * unigram_prob

def comp_perplexity(corpus, weights):
    lambda1, lambda2, lambda3 = weights
    log_sum = 0.0
    ngram_count = 0
    
    for para in corpus:
        tokens = para.split()
        for i in range(len(tokens) - 2):
            w1, w2, w3 = tokens[i], tokens[i+1], tokens[i+2]
            prob = compute_interpolated_prob(w1, w2, w3, (lambda1, lambda2, lambda3))
            
            if prob <= 0:
                prob = 1e-15 
            
            log_sum += np.log(prob)
            ngram_count += 1
    
    return np.exp(-log_sum / ngram_count) if ngram_count > 0 else float('inf')

def optimal_weights(validation_texts):
    optimal_perplexity = float('inf')
    optimal_weights = (0, 0, 0)
    
    for lambda1 in tqdm(np.linspace(0, 1, 10)):
        for lambda2 in tqdm(np.linspace(0, 1 - lambda1, 10)):
            lambda3 = 1 - lambda1 - lambda2
            perplexity = comp_perplexity(validation_texts, (lambda1, lambda2, lambda3))
            
            if perplexity < optimal_perplexity:
                optimal_perplexity = perplexity
                optimal_weights = (lambda1, lambda2, lambda3)
    
    return optimal_weights, optimal_perplexity

optimal_lambdas, min_perplexity = optimal_weights(validation_set.Text)
print(f"Optimal Weights: {optimal_lambdas}, Minimum Perplexity: {min_perplexity}")

test_perplexity = comp_perplexity(test_set['Text'], optimal_lambdas)
print(f"Final Test Perplexity: {test_perplexity}")


100%|██████████| 10/10 [00:43<00:00,  4.34s/it]


Optimal Weights: (np.float64(0.8888888888888888), np.float64(0.11111111111111116), np.float64(0.0)), Minimum Perplexity: 1018.31724069965
Final Test Perplexity: 997.0694520648213
